In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum
from pyspark.sql.types import StructType, StructField, StringType

spark = SparkSession.builder.appName("Sales Revenue per Continent").getOrCreate()

sales_df = spark.read.csv("all_countries_sales_data.csv", header=True, inferSchema=True)

sales_df = sales_df.withColumn("Revenue", col("UnitsSold") * col("UnitPrice"))

sales_df = sales_df.drop("continent")

country_continent_data = [
    ("USA", "North America"), ("Canada", "North America"), ("Mexico", "North America"),
    ("Brazil", "South America"), ("Argentina", "South America"),
    ("Germany", "Europe"), ("France", "Europe"), ("UK", "Europe"),
    ("India", "Asia"), ("China", "Asia"), ("Japan", "Asia"),
    ("Australia", "Oceania"), ("South Africa", "Africa"), ("Egypt", "Africa")
]

schema = StructType([
    StructField("Country", StringType(), True),
    StructField("Continent", StringType(), True)
])

continent_df = spark.createDataFrame(country_continent_data, schema=schema)

merged_df = sales_df.join(continent_df, sales_df["country"] == continent_df["Country"], how="left") .drop("Country")  


merged_df = merged_df.na.fill({"Continent": "Unknown"})


revenue_by_continent = merged_df.groupBy("Continent").agg(
    spark_sum("Revenue").alias("TotalRevenue")
)

revenue_by_continent.show()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/22 18:18:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-------------+------------------+
|    Continent|      TotalRevenue|
+-------------+------------------+
|       Europe|3301564.5800000005|
|       Africa|2384407.6600000006|
|North America| 2922833.039999999|
|South America|2228105.8800000004|
|      Oceania| 969209.2999999998|
|         Asia|        2952033.76|
+-------------+------------------+

